# Day 11

## Part 1

In [1]:
import functools
from math import floor

In [2]:

class Monkey: 
    def __init__(self, m_in):
        self.num = m_in['num']
        self.items = m_in['starting_items']
        self.operation = m_in['operation']
        self.test = m_in['test']
        self.inspections = 0
        
    def __repr__(self):
        out = str(self.num)
        out += ': '
        for item in self.items: 
            out += str(item) + ' '
        out += '\n'
        return(out)
        

class MonkeyGroup: 
    def __init__(self): 
        self.monkeys = {}
            
    def __repr__(self):
        out = 'MonkeyGroup\n'
        for key, value in self.monkeys.items(): 
            out += value.__repr__()
        return(out)
        
    def add_monkey(self, m): 
        self.monkeys[m.num] = m
    
    def pass_item(self, monkey_num, verbose=False): 
        if len(self.monkeys[monkey_num].items) > 0: 
            self.monkeys[monkey_num].inspections += 1
            
            item = self.monkeys[monkey_num].items.pop(0)
            new_item = self.monkeys[monkey_num].operation(item)
            new_item = floor(new_item / 3)
            new_monkey = self.monkeys[monkey_num].test(new_item)
            self.monkeys[new_monkey].items.append(new_item)
            if verbose: 
                print('---')
                print(item)
                print(new_item)
                print(new_monkey)
                print('---')
            return 1
        else: 
            if verbose: 
                print('No more items!')
            return 0

In [3]:
# parse monkeys
def monkey_setup(input_file):         
    f = open(input_file, 'r')

    monkeys = MonkeyGroup()
    monkey_input = {}
    parsing_test = 0
    test_input = {}

    while True: 
        line = f.readline()
        if not line:
            out_true = test_input['true']
            out_divis = test_input['divis_by']
            out_false = test_input['false']
            monkey_input['test'] = functools.partial(lambda x, out_true=out_true, out_divis=out_divis, out_false=out_false: out_true if ((x % out_divis) == 0) else out_false)

            break
        if parsing_test == 1: 
            if line == '\n': 
                parsing_test = 0

                out_true = test_input['true']
                out_divis = test_input['divis_by']
                out_false = test_input['false']
                monkey_input['test'] = functools.partial(lambda x, out_true=out_true, out_divis=out_divis, out_false=out_false: out_true if ((x % out_divis) == 0) else out_false)

                test_input = {}
            elif 'true' in line: 
                test_input['true'] = int(line[:-1].split(' ')[-1])
            else: 
                test_input['false'] = int(line[:-1].split(' ')[-1])
        elif 'Monkey' in line: 
            if not monkey_input == {}:
                m = Monkey(monkey_input)
                monkeys.add_monkey(m)
            monkey_input = {}
            monkey_input['num'] = int(line[-3:-2])
        elif 'Starting items:' in line: 
            monkey_input['starting_items'] = [int(x) for x in line[18:-1].split(', ')]
            # monkey_input['starting_items'].reverse()
        elif 'Operation:' in line: 
            split_line = line[13:-1].split(' ')
            v2 = split_line[4]
            if split_line[3] == '*': 
                if v2 == 'old': 
                    monkey_input['operation'] = lambda x: x * x
                else: 
                    v2 = int(v2)
                    monkey_input['operation'] = functools.partial(lambda x, v2=v2: x * v2)
            elif split_line[3] == '+': 
                if v2 == 'old': 
                    monkey_input['operation'] = lambda x: x + x
                else: 
                    v2 = int(v2)
                    monkey_input['operation'] = functools.partial(lambda x, v2=v2: x + v2)
            else: 
                print('You forgot to consider something')
        elif 'Test' in line: 
            parsing_test = 1
            test_input['divis_by'] = int(line[8:-1].split(' ')[2])

    m = Monkey(monkey_input)
    monkeys.add_monkey(m)
    
    return(monkeys)

In [4]:
monkeys = monkey_setup('example11')
print(monkeys)
for round in range(20):
    for i in range(len(monkeys.monkeys)): 
        while True: 
            out = monkeys.pass_item(i)
            if out == 0: 
                break
print(monkeys)
print([m.inspections for key, m in monkeys.monkeys.items()])

MonkeyGroup
0: 79 98 
1: 54 65 75 74 
2: 79 60 97 
3: 74 

MonkeyGroup
0: 10 12 14 26 34 
1: 245 93 53 199 115 
2: 
3: 

[101, 95, 7, 105]


In [5]:
101*105

10605

Now to do the real thing!

In [6]:
monkeys = monkey_setup('input11')
print(monkeys)
for round in range(20):
    for i in range(len(monkeys.monkeys)): 
        while True: 
            out = monkeys.pass_item(i)
            if out == 0: 
                break
print(monkeys)
print([m.inspections for key, m in monkeys.monkeys.items()])


MonkeyGroup
0: 93 54 69 66 71 
1: 89 51 80 66 
2: 90 92 63 91 96 63 64 
3: 65 77 
4: 76 68 94 
5: 86 65 66 97 73 83 
6: 78 
7: 89 57 59 61 87 55 55 88 

MonkeyGroup
0: 
1: 2 2 2 2 2 2 2 2 2 2 2 2 2 
2: 
3: 4 4 4 4 4 4 4 4 4 4 4 4 956 
4: 
5: 6 6 6 6 6 6 6 6 6 6 
6: 
7: 

[14, 239, 22, 238, 25, 231, 236, 246]


In [7]:
246 * 239

58794

## Troubleshooting lambda functions

The first iteration of these lambda functions that I wrote didn't work, so I got ChatGPT to help me solve it! I asked it: 

```
Hey ChatGPT! I'm trying to solve a coding problem where I assign lambda functions to objects. However, I'm running into a problem because I'm using a variable to assign values to the lambda functions (see the example below) and the value of the variable is changing; I want it to be fixed. How can I solve this problem?
```

In [8]:
class Tester: 
    def __init__(self, operation): 
        self.operation = operation

testers = {}
for i in range(5): 
    op = lambda x: x + i
    testers[i] = Tester(op)
    
[x.operation(4) for x in testers.values()]

[8, 8, 8, 8, 8]

And it responded: 
```
In the example you provided, the problem is that the value of the variable i changes in each iteration of the loop, and the lambda function defined within the loop captures the value of i at the time it is defined. As a result, all of the lambda functions will have the same value of i when they are called, which is the value of i in the final iteration of the loop.

To fix this problem, you can use the functools.partial function from the functools module to create a new function that has a fixed value for the parameter i. Here's how you can do it:
```

In [9]:
import functools

class Tester: 
    def __init__(self, operation): 
        self.operation = operation

testers = {}
for i in range(5): 
    op = functools.partial(lambda x, i=i: x + i)
    testers[i] = Tester(op)
    
[x.operation(4) for x in testers.values()]


[4, 5, 6, 7, 8]

The functools.partial function creates a new function that takes a single argument, x, and has a fixed value for the parameter i. This new function is then passed to the Tester constructor and assigned to the operation attribute. When the lambda function is called, it will use the fixed value of i that was specified when the function was created.

I have no idea if the explanation it gave is correct but the code works. 

## Part 2

The problem with Part 2 is that you no longer get to divide the "worry level" of the item by 3, which causes the values to balloon rapidly out of control, such that simulating even 1000 passes (which is trivial in Part 1) is difficult in part 2. However, due to the properties of mods, we can simply mod the values by a lowest common divisor which will keep the values within a reasonable range. 

For instance, if the rules for monkeys 1, 2, and 3 are 'divisible by 12', 'divisible by 29', and 'divisible by 21', then I can just multiple 12, 29, and 21 together, mod every monkey item by that, and still retain accurate calculations even across thousands of iterations. Yay!

In [22]:

class Monkey: 
    def __init__(self, m_in):
        self.num = m_in['num']
        self.items = m_in['starting_items']
        self.operation = m_in['operation']
        self.test = m_in['test']
        self.inspections = 0
        self.divisor = m_in['divisor']
        
    def __repr__(self):
        out = str(self.num)
        out += ': '
        for item in self.items: 
            out += str(item) + ' '
        out += '\n'
        return(out)
        

class MonkeyGroup: 
    def __init__(self): 
        self.monkeys = {}
        self.lcdivisor = 1
            
    def __repr__(self):
        out = 'MonkeyGroup\n'
        for key, value in self.monkeys.items(): 
            out += value.__repr__()
        return(out)
        
    def add_monkey(self, m): 
        self.monkeys[m.num] = m
        self.lcdivisor = self.lcdivisor*m.divisor
    
    def pass_item(self, monkey_num, verbose=False): 
        if len(self.monkeys[monkey_num].items) > 0: 
            self.monkeys[monkey_num].inspections += 1
            
            item = self.monkeys[monkey_num].items.pop(0)
            new_item = self.monkeys[monkey_num].operation(item)
            new_monkey = self.monkeys[monkey_num].test(new_item)
            new_item = new_item % self.lcdivisor
            self.monkeys[new_monkey].items.append(new_item)
            if verbose: 
                print('---')
                print(item)
                print(new_item)
                print(new_monkey)
                print('---')
            return 1
        else: 
            if verbose: 
                print('No more items!')
            return 0

In [23]:
# parse monkeys
def monkey_setup(input_file):         
    f = open(input_file, 'r')

    monkeys = MonkeyGroup()
    monkey_input = {}
    parsing_test = 0
    test_input = {}

    while True: 
        line = f.readline()
        if not line:
            out_true = test_input['true']
            out_divis = test_input['divis_by']
            out_false = test_input['false']
            monkey_input['divisor'] = test_input['divis_by']
            monkey_input['test'] = functools.partial(lambda x, out_true=out_true, out_divis=out_divis, out_false=out_false: out_true if ((x % out_divis) == 0) else out_false)

            break
        if parsing_test == 1: 
            if line == '\n': 
                parsing_test = 0

                out_true = test_input['true']
                out_divis = test_input['divis_by']
                out_false = test_input['false']
                monkey_input['divisor'] = test_input['divis_by']
                monkey_input['test'] = functools.partial(lambda x, out_true=out_true, out_divis=out_divis, out_false=out_false: out_true if ((x % out_divis) == 0) else out_false)

                test_input = {}
            elif 'true' in line: 
                test_input['true'] = int(line[:-1].split(' ')[-1])
            else: 
                test_input['false'] = int(line[:-1].split(' ')[-1])
        elif 'Monkey' in line: 
            if not monkey_input == {}:
                m = Monkey(monkey_input)
                monkeys.add_monkey(m)
            monkey_input = {}
            monkey_input['num'] = int(line[-3:-2])
        elif 'Starting items:' in line: 
            monkey_input['starting_items'] = [int(x) for x in line[18:-1].split(', ')]
            # monkey_input['starting_items'].reverse()
        elif 'Operation:' in line: 
            split_line = line[13:-1].split(' ')
            v2 = split_line[4]
            if split_line[3] == '*': 
                if v2 == 'old': 
                    monkey_input['operation'] = lambda x: x * x
                else: 
                    v2 = int(v2)
                    monkey_input['operation'] = functools.partial(lambda x, v2=v2: x * v2)
            elif split_line[3] == '+': 
                if v2 == 'old': 
                    monkey_input['operation'] = lambda x: x + x
                else: 
                    v2 = int(v2)
                    monkey_input['operation'] = functools.partial(lambda x, v2=v2: x + v2)
            else: 
                print('You forgot to consider something')
        elif 'Test' in line: 
            parsing_test = 1
            test_input['divis_by'] = int(line[8:-1].split(' ')[2])

    m = Monkey(monkey_input)
    monkeys.add_monkey(m)
    
    return(monkeys)

In [26]:
monkeys = monkey_setup('example11')
print(monkeys.lcdivisor)
for round in range(20):
    for i in range(len(monkeys.monkeys)): 
        while True: 
            out = monkeys.pass_item(i)
            if out == 0: 
                break
print([m.inspections for key, m in monkeys.monkeys.items()])

monkeys = monkey_setup('example11')
for round in range(10000):
    for i in range(len(monkeys.monkeys)): 
        while True: 
            out = monkeys.pass_item(i)
            if out == 0: 
                break
print([m.inspections for key, m in monkeys.monkeys.items()])

96577
[99, 97, 8, 103]
[52166, 47830, 1938, 52013]


In [27]:
52166*52013

2713310158

Now to do the real thing!

In [28]:
monkeys = monkey_setup('input11')
print(monkeys)
for round in range(10000):
    for i in range(len(monkeys.monkeys)): 
        while True: 
            out = monkeys.pass_item(i)
            if out == 0: 
                break
print(monkeys)
print([m.inspections for key, m in monkeys.monkeys.items()])


MonkeyGroup
0: 93 54 69 66 71 
1: 89 51 80 66 
2: 90 92 63 91 96 63 64 
3: 65 77 
4: 76 68 94 
5: 86 65 66 97 73 83 
6: 78 
7: 89 57 59 61 87 55 55 88 

MonkeyGroup
0: 
1: 6344168 3992279 9205499 1711559 6169469 6169469 672689 2707589 8667959 1283115 
2: 9205490 1757960 3204830 8496080 8812280 
3: 8015209 6158299 4868101 3997429 8205354 1164804 8750850 1717950 1751814 8170011 
4: 
5: 557301 6158291 3067181 8015201 6120143 1933961 9030016 7027246 7294486 3527932 3527932 
6: 
7: 

[598, 142046, 66128, 141864, 36135, 76332, 141471, 141605]


In [30]:
142046*141864

20151213744